# Euro Millions

By ***Rok Medves***

Remarks on dataset can be found in ./datasets/README.md


## Loading the dataset

In [204]:
import pandas as pd
import numpy as np
import itertools
import matplotlib.pyplot as plt

# Load in the data -- could be automated to just pull the data from the webpage
euromillions_draws   = pd.read_csv('./datasets/euro-millions-draws.csv',   index_col=0)
euromillions_winners = pd.read_csv('./datasets/euro-millions-winners.csv', index_col=0)
euromillions_sales   = pd.read_csv('./datasets/euro-millions-sales.csv', usecols=[4,])

euromillions_sales = euromillions_sales.set_index(euromillions_winners.index)

# clean column names
for dataset in (euromillions_draws, euromillions_winners, euromillions_sales):
    dataset.columns = [ col.strip() for col in dataset.columns]

# only temporary:
# 
# On sep 2016 the number of winning numbers changed for the last time (see ./datasets/README.md)
# This flag selects only data after that date
select_only_pre_sep_2016_data = True


## Dataset Cleaning and Processing

In [205]:
# ---------------------------- handing NaN values in sales dataset -----------------------------------------

# there is a N/A in the sales dataset 
# find it and set the sales value to the expected total.
# This needs to be done manually because of spaces 
# invalidating the pd.isnull() & pd.isna() functions
#
# the expected number of sales should roughly be 13 * the
# total number of winners
na_loc = euromillions_sales.Sales.str.strip() == 'N/A'
na_ind = euromillions_sales[na_loc].index[0]

# Replace the `N/A` with a values reflecting the sales/winners mean. 
# it's quick and dirty, but it gets the ratio right
euromillions_sales.loc[na_ind] = euromillions_sales.Sales.drop(index = na_ind).astype(float).mean()/euromillions_winners.Total.mean() * euromillions_winners.Total.loc[na_ind]
euromillions_sales = euromillions_sales.astype(float)

# ---------------------------- Filling in missing values for number of winners -----------------------------------------
# When the category with 2 matching numbers is introduced, the dataset gains a column. 
# The following code amends the missing values by setting the number of winners in the group `2` to 0
# Admittedly, this could be done better!
# TODO: Find a better way of filling in missing values without messing up the predictive power of the model

#Splitting the dataset into a NaN region and a non-Nan region
winners_nan, winners_nonan = euromillions_winners[pd.isna(euromillions_winners.Total) == True], euromillions_winners[pd.isna(euromillions_winners.Total) == False],

# Reindexing things
winners_nan.loc[:, '2'], winners_nan.loc[:, 'Total'] = winners_nan.loc[:, 'Total'].copy(), winners_nan.loc[:, '2'].copy()

# Replacing all NaN with 0
winners_nan = winners_nan.fillna(0)

# Combining everything
euromillions_winners = pd.concat([winners_nonan, winners_nan])

# Now we can actually combine the whole dataset 
euromillions = pd.concat([euromillions_draws, euromillions_winners, euromillions_sales], axis = 1)

# ---------------------------- Validate the cleaning procedure -----------------------------------------

# Validate the clean
summed_winnings = euromillions.loc[:, '5+2':'2'].apply(np.sum, axis = 'columns')
total_validation = pd.concat([summed_winnings, euromillions.Total], axis = 1).rename(columns={0: 'true', 'Total' : 'dataset'})

def validate_cleaning(df : pd.DataFrame):
    """
    Validates the cleaning procedure by
    checking whether the total number of
    winners is consistent accross the dataset

    Parameters
    ----------
    df : pandas.Dataframe
        The dataframe to validate the cleaning of

    Returns
    -------
    bool
        Whether the dataset is lean or not
    """

    clean = True

    df_notnan = df[pd.notna(df.dataset)]
    df_diff    = df_notnan.apply(lambda df: df.dataset - df.true, axis = 'columns')
    if df_diff.sum() != 0.0:
        print("The original total columns were wrong!")
        print(df_notnan[df_diff != 0])

        clean = False

    return clean

if validate_cleaning(total_validation): print("Dataset cleaned accordingly")

# trim extra values; flag set at the top, when loading the data
def trim_by_date(df, do_select = False) -> pd.DataFrame:
    """ 
    Trims the dataset df with respect to the date.
    Note that this date is hard-coded to match when the 
    euromillions changed the range of their lucky numbers from 11 to 12

    Parameters
    ----------
    df : pandas.Dataframe
        The dataframe we trim
    do_select : bool
        whether or not to do the selection
    
    Returns
    -------
    pandas.Dataframe
        The trimmed array

    """
    min_index = 0
    if do_select:
        date_mask = (euromillions.DD == 27) & (euromillions.MMM == 'Sep') & (euromillions.YYYY == 2016)
        min_index = euromillions[date_mask].index[0]
    
    print("additionally removing {0} entries from dataset".format(min_index))
    return df.loc[df.index >= min_index]

euromillions = trim_by_date(euromillions, select_only_pre_sep_2016_data)




Dataset cleaned accordingly
additionally removing 941 entries from dataset


/usr/lib/python3/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


## Feature Engineering

I speculate that people like to bet on numbers that are either
- dates (birthdays, anniversaries, etc.)
- 'nice' numbers, such as 3, 7, 13

To leverage this, new features are engineered below 

In [206]:
def is_date(c : pd.Series) -> bool:
    """ 
    Checks whether the row c contains
    a valid date. E.g.
    5. 10.  -- > True
    if no such dates are found, it returns false

    Parameters
    ----------
    c : pandas.Series
        the rows that will be checked
    
    Returns
    -------
    bool
        the truth value of whether the row 
        contains a valid date
    """
    
    day = False
    month = False
    for lab, i in c.iteritems():
        if 'N' not in lab: continue

        if not month:
            if i <= 12: month = True
        elif not day:
            if i <= 31: day = True
        else:
            break
    return day and month

def is_this_year(c : pd.Series) -> bool:
    """ 
    Checks whether the row c contains
    the same year that the draw happend

    Parameters
    ----------
    c : pandas.Series
        the rows that will be checked
    
    Returns
    -------
    bool
        the truth value of whether the row 
        contains the current year
    """
    first = 20
    second = c['YYYY']-2000

    f, s = False, False
    for lab, i in c.iteritems():
        if 'N' not in lab: continue

        if   i == first:  f = True
        elif i == second: s = True
    
    return f and s

def is_post_2000(c : pd.Series) -> bool:
    """ 
    Checks whether the row c contains
    the a valid year past the year 2000

    Parameters
    ----------
    c : pandas.Series
        the rows that will be checked
    
    Returns
    -------
    bool
        the truth value of whether the row 
        contains a year after 2000
    """
    first = 20
    second = c['YYYY']-2000

    f, s = False, False
    for lab, i in c.iteritems():
        if 'N' not in lab: continue

        if   i == 20:  f = True
        elif i <= second: s = True
    
    return f and s

def get_lucky_numbers(c : pd.Series) -> int:
    """ 
    Checks how many nice numbers the row c contains

    Parameters
    ----------
    c : pandas.Series
        the rows that will be checked
    
    Returns
    -------
    int
        The number of nice numbers in the data
    """

    # these lucky numbers are directly from the web 
    # Supposedlty some people really believe in these numbers
    lucky_numbers = [1, 3, 7, 9, 13, 15, 21, 25, 31, 33, 37, 43, 49] 
    total = 0
    for n in lucky_numbers:
        total += n in c.values
    return total


In [207]:
N_numbers = euromillions.loc[:,'YYYY':'N5']
L_numbers = euromillions.loc[:, ['L1', 'L2']]

# ---------------------------- date-based-features -----------------------------------------
euromillions["is date"]      = N_numbers.apply(is_date, axis = 1)
euromillions["is post 2000"] = N_numbers.apply(is_post_2000, axis = 1)
euromillions["is this year"] = N_numbers.apply(is_this_year, axis = 1)
# ---------------------------- lucky-numbers-based procedures -----------------------------------------
euromillions["lucky numbers"]= N_numbers.apply(get_lucky_numbers, axis = 1)
euromillions["lucky lucky numbers"]= L_numbers.apply(get_lucky_numbers, axis = 1)
euromillions["has lucky"]    = euromillions["lucky numbers"]>0
euromillions["has lucky lucky"]    = euromillions["lucky lucky numbers"]>0

# ---------------------------- drop unwanted values -----------------------------------------
euromillions  = euromillions.drop(columns = ['Day', 'DD', 'MMM', 'YYYY', 'Wins'])

## Defining the target variable

The idea here is the following:
- Let *k* be the class of winnings that we consider i.e. (5+2, 5+1, ..., 2+1, 2)
- Let *f_w,k* be the fraciton of people who have won in a single category *k*. So, *f_w,k = #sales/#(winners in calss k)*
- Let *f_p,k* be the fraction of the prize for a category *k*. These numbers are fixed and are encoded below, and do indeed match with the data
- Let *Pr[k | win]* be the probability that you win the prize for group *k*. NB: *Pr[k] = Pr[k | win] * Pr[win] = Pr[k | win] * 1/13* 

For a given drawn number, one can write the average winnings as
~~~
E[winnings | number] = Jackpot * sum_k Pr[k | win] * Pr[win] * f_p,k * f_w,k
~~~

The only parameter in this equation which depends on the number being drawn is *f_w,k* and this is what we would like to learn!
We define
~~~
Score = S = E[winnings | number] / E[winnings],
~~~
where *E[winnings]* is the aveage of the winnings throughout all the numbers.

This should split numbers into 2 regions: 
1. Numbers with score >1:  These numbers are better than average
2. Numbers with score <=1: These numbers are worse than average

The below table describes the proportions of the prize fund allocated to the particular winning group *(N, L)* [copied from above]
| Winning group (N, L) | 5+2 | 5+1 | 5+0 | 4+2 | 4+1 | 3+2 | 4+0 | 2+2 | 3+1 | 3+0 | 1+2 | 2+1 | 2+0 |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| % of prize fund | 50 | 2.61 | 0.61 | 0.19 | 0.35 | 0.37 | 0.26 | 1.30 | 1.45 | 2.70 | 3.27 | 10.30 | 16.59 |

In [208]:
# f_p,k from above
win_frac = {5: {0: 0.0061, 1: 0.0261, 2: 0.5000},
            4: {0: 0.0026, 1: 0.0035, 2: 0.0019},
            3: {0: 0.0270, 1: 0.0145, 2: 0.0037},
            2: {0: 0.1659, 1: 0.1030, 2: 0.0130},
            1: {2: 0.0327}}

from scipy.special import binom
def prob_NL_analyt(N: int, L : int) -> float:
    """ Computes the probability to get N normal and L lucky numbers right in a draw

    Parameters
    ----------
    N : int
        Number of correct drawn normal numbers
    L : int
        Number of correct drawn lucky number

    Returns
    -------
        Probability Pr[win in category N+L]
    """
    return binom(5, N) * binom(45, 5-N) * binom(2, L) * binom(10, 2-L) / (binom(50, 5) * binom(12, 2))

# do this row per row
def score_numbers(row: pd.Series) -> float:
    """
    Given a row from the euromillions dataset, generate 
    a score for the given number with the recipe above.
    If called with pd.apply(), this function generates a
    whole pd.Series().

    Parameters
    ----------
    row: pd.Series
        Row of the data to score

    Returns
    -------
        average winnigs for that number
    """
    avg_win = 0
    # loop through the possible winning groups
    for N, L in itertools.product(range(1, 6), range(0, 3)):
        nl_tag = str(N)+ ("+"+str(L) if L!=0 else "")
        if nl_tag in row.index:
            pr_nl_win   = prob_NL_analyt(N, L) # Pr[N,L] = Pr[N,L | win] * 1/13 from above
            nl_win_frac = win_frac[N][L]       # f_p,k=(N,L)
            num_winners = row[nl_tag]
            num_sales   = row['Sales']

            f_w_nl      = (0 if num_winners == 0 else num_sales/num_winners) # f_w,k=(N,L)

            avg_win += pr_nl_win * nl_win_frac * f_w_nl
            # Diagnostic print 
            # print("k = {0}\t\tPr[N,L] = {1:.4f}, f_w,k = {2:.4f}, n_k = {3}\t\tE[win] = {4}".format(nl_tag, pr_nl_win, nl_win_frac, num_winners, avg_win))
    return avg_win

def score_dataset(df : pd.DataFrame) -> pd.DataFrame:
    """
    Scores the datset by assigning 'Score' and 'Class' columns

    Parameters
    ----------
    df: pd.DataFrame
        The DataFrame to be scored 
    Returns
    -------
    pd.DataFrame
        The scored dataframe
    """
    # generate the average winnings for each number
    scores = df.apply(score_numbers, axis = 'columns')
    # devide by the mean of each, thus getting the final score
    scores = scores / scores.mean()

    # incorporate that into the euromillions dataset
    df = pd.concat([df, scores], axis = 1).rename(columns={0: 'Score'})

    # classify each datapoint into a good or bad data point
    good_bad_class = df.apply(lambda row: 1 if row['Score'] > 1 else 0, axis = 1)
    df = pd.concat([df, good_bad_class], axis = 1).rename(columns={0: 'Class'})

    return df

In [209]:
euromillions = score_dataset(euromillions)
best = euromillions[euromillions.Score == euromillions.Score.max()].iloc[0]
worst= euromillions[euromillions.Score == euromillions.Score.min()].iloc[0]
# print(pd.DataFrame([best, worst], index = ["best", "worst"]))

In [210]:
euromillions

,N1,N2,N3,N4,N5,L1,L2,Jackpot,5+2,5+1,...,Sales,is date,is post 2000,is this year,lucky numbers,lucky lucky numbers,has lucky,has lucky lucky,Score,Class
No.,,,,,,,,,,,,,,,,,,,,,
1611,12,29,33,37,47,5,10,14937900,0,0,...,5842047.0,True,False,False,2,0,True,False,0.170171,0
1610,7,23,34,42,48,1,3,90371760,0,1,...,60482734.0,True,False,False,1,2,True,True,1.012454,1
1609,21,22,34,41,49,2,7,77415556,0,0,...,46454399.0,False,False,False,2,1,True,True,1.048413,1
1608,8,23,27,42,45,1,9,69376023,0,1,...,53000419.0,True,False,False,0,2,False,True,0.938640,0
1607,24,26,38,43,46,2,3,55753429,0,1,...,40015726.0,False,False,False,1,1,True,True,1.097452,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
945,5,12,20,24,37,4,11,153361048,0,3,...,84831239.0,True,True,False,1,0,True,False,0.900919,0
944,2,12,20,27,30,9,11,140104641,0,2,...,98334429.0,True,True,False,0,1,False,True,0.978729,0
943,4,7,14,34,38,4,11,124041722,0,0,...,70457205.0,True,False,False,1,0,True,False,0.979008,0
